In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
0_youtube_audio_worker_v3_progress.py
(No progress → Progress bars added + duration auto-repair stays)

- assignments_audio_<RUN_ID>.csv を読み、指定 shard の video_id だけ処理
- 音声: HLSを避けて m4a(itag=140) 等の non-HLS を優先 (tv→android→web)
- チャット: live_chat / live_chat_replay を json3（なければ json）で取得
- 中断→再実行で既に終わったものは skip（音声だけOKならチャットだけ再開）#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""


from __future__ import annotations
import os, re, sys, csv, glob, time, traceback, json, subprocess, shutil
from typing import List, Dict, Tuple, Optional

# ========== ユーザー設定（ここを編集） ==========
BASE_ROOT = "先ほどと同じディレクトリを入力してください、絶対パスで"
RUN_ID    = "20250910_104532"  # 例: プランナー出力の run_id
SHARD_IDS = [0]                # 実行する shard のリスト
PARALLEL  = False              # True で並列（必要時のみ推奨）

COOKIES   = "ここにcookieのファイルの絶対パスを入れてください。google chromeでGet cookies.txt LOCALLYの拡張機能を使うとダウンロードできます。"
COOKIE_MAX_AGE_DAYS: Optional[int] = 7    # None=無効 / 数値=古い場合に警告

# ffmpeg/ffprobe のパス（空なら PATH 検索）
FFMPEG_PATH  = "/usr/local/bin/ffmpeg"
FFPROBE_PATH = "/usr/local/bin/ffprobe"

# SABR対策などの穏やかなレート
SLEEP_BETWEEN_VIDEOS = 0.3
BASE_SLEEP_REQUESTS  = 1.0
BASE_MAX_SLEEP_REQ   = 2.5
RATE_LIMIT_BYTES     = 6 * 1024 * 1024  # 6 MiB/s

# 長さチェック/修復の閾値
DURATION_CHECK_ENABLED   = True
DURATION_TOLERANCE_RATIO = 1.25  # 実際長が期待長の 1.25 倍を超えたら修復
DURATION_TOLERANCE_ABS   = 300   # または 300 秒以上ズレたら修復
# ============================================

try:
    import yt_dlp
    from yt_dlp.utils import DownloadError, ExtractorError
except Exception:
    print("yt-dlp が見つかりません。インストール:  python3 -m pip install -U yt-dlp", file=sys.stderr)
    raise


# ---------------- path utils ----------------
def run_root(run_id: str) -> str:
    return os.path.join(BASE_ROOT, run_id)

def ensure_dirs(base: str) -> Dict[str, str]:
    sub = {
        "audio": os.path.join(base, "audio"),
        "chat":  os.path.join(base, "chat"),
        "meta":  os.path.join(base, "meta"),
        "logs":  os.path.join(base, "logs"),
        "tmp":   os.path.join(base, "tmp"),
        "mani":  os.path.join(base, "manifests"),
    }
    for p in sub.values():
        os.makedirs(p, exist_ok=True)
    return sub

def normalize_to_url(s: str) -> str:
    return f"https://www.youtube.com/watch?v={s}" if re.fullmatch(r"^[\w-]{11}$", s) else s


# ---------------- helpers ----------------
def already_has_audio(audio_dir: str, vid: str) -> bool:
    for ext in ("m4a", "mp4", "webm"):
        if os.path.exists(os.path.join(audio_dir, f"{vid}.{ext}")):
            return True
    return False

def already_has_chat(chat_dir: str, vid: str) -> bool:
    for name in (
        f"{vid}.live_chat.json3",
        f"{vid}.live_chat.json",
        f"{vid}.live_chat_replay.json3",
        f"{vid}.live_chat_replay.json",
    ):
        if os.path.exists(os.path.join(chat_dir, name)):
            return True
    return False

def move_info_json_to_meta(audio_dir: str, meta_dir: str):
    for path in glob.glob(os.path.join(audio_dir, "*.info.json")):
        name = os.path.basename(path)
        dst = os.path.join(meta_dir, name)
        try:
            if os.path.exists(dst):
                os.remove(dst)
            os.replace(path, dst)
        except Exception as e:
            print("[WARN] move info.json failed:", e)

def cookie_is_fresh(path: str, max_days: Optional[int]) -> bool:
    if max_days is None:
        return True
    try:
        mtime = os.path.getmtime(path)
    except OSError:
        return False
    age_days = (time.time() - mtime) / 86400.0
    return age_days <= max_days

def which(cmd: str) -> Optional[str]:
    if not cmd:
        return shutil.which(cmd)  # None
    return cmd if os.path.exists(cmd) else shutil.which(cmd)

def ffprobe_duration_seconds(path: str, ffprobe_path: str) -> Optional[float]:
    if not os.path.exists(path):
        return None
    ffprobe = which(ffprobe_path) or shutil.which("ffprobe")
    if not ffprobe:
        return None
    try:
        out = subprocess.check_output(
            [ffprobe, "-v", "error", "-show_entries", "format=duration",
             "-of", "default=nw=1:nk=1", path],
            stderr=subprocess.STDOUT
        )
        s = out.decode("utf-8", "ignore").strip()
        return float(s) if s else None
    except Exception:
        return None

def read_expected_duration_from_infojson(info_path: str) -> Optional[float]:
    if not os.path.exists(info_path):
        return None
    try:
        with open(info_path, "r", encoding="utf-8") as f:
            j = json.load(f)
        d = j.get("duration")
        if isinstance(d, (int, float)):
            return float(d)
        ds = j.get("duration_string") or ""
        if ds:
            parts = [float(x) for x in ds.split(":")]
            if len(parts) == 3:
                return parts[0]*3600 + parts[1]*60 + parts[2]
            if len(parts) == 2:
                return parts[0]*60 + parts[1]
        return None
    except Exception:
        return None

def find_audio_file(audio_dir: str, vid: str) -> Optional[str]:
    for ext in ("m4a", "mp4", "webm"):
        p = os.path.join(audio_dir, f"{vid}.{ext}")
        if os.path.exists(p):
            return p
    return None

def find_infojson(audio_dir: str, meta_dir: str, vid: str) -> Optional[str]:
    a = os.path.join(audio_dir, f"{vid}.info.json")
    if os.path.exists(a):
        return a
    m = os.path.join(meta_dir, f"{vid}.info.json")
    if os.path.exists(m):
        return m
    return None

def needs_repair(expected: Optional[float], actual: Optional[float]) -> bool:
    if expected is None or actual is None:
        return False
    if actual > expected * DURATION_TOLERANCE_RATIO:
        return True
    if abs(actual - expected) > DURATION_TOLERANCE_ABS and actual > expected:
        return True
    return False

def try_repair_container_ffmpeg(src_path: str, dst_path: str, ffmpeg_path: str) -> Tuple[bool, str]:
    ffmpeg = which(ffmpeg_path) or shutil.which("ffmpeg")
    if not ffmpeg:
        return False, "ffmpeg not found"
    cmd = [
        ffmpeg, "-y", "-loglevel", "error",
        "-i", src_path,
        "-c", "copy",
        "-movflags", "+faststart",
        "-fflags", "+bitexact",
        "-avoid_negative_ts", "make_zero",
        dst_path,
    ]
    try:
        subprocess.check_call(cmd)
        return True, "remuxed"
    except subprocess.CalledProcessError as e:
        return False, f"ffmpeg failed({e.returncode})"
    except Exception as e:
        return False, repr(e)


# --------- formatting utils for progress ----------
def _fmt_bytes(n: Optional[float]) -> str:
    if n is None:
        return "?"
    units = ["B","KiB","MiB","GiB","TiB"]
    i = 0
    v = float(n)
    while v >= 1024 and i < len(units)-1:
        v /= 1024.0
        i += 1
    return f"{v:.2f}{units[i]}"

def _fmt_time(t: Optional[float]) -> str:
    if t is None:
        return "?:??"
    t = int(max(0, t))
    h, r = divmod(t, 3600)
    m, s = divmod(r, 60)
    if h > 0:
        return f"{h:d}:{m:02d}:{s:02d}"
    return f"{m:d}:{s:02d}"

def _pct(dl: Optional[float], total: Optional[float]) -> Optional[float]:
    if dl is None or total is None or total <= 0:
        return None
    return min(100.0, max(0.0, 100.0 * dl / total))


# -------- SABR-aware logging --------
_SABR_HINT_PRINTED_FOR_SHARDS = set()

class DualLogger:
    """yt-dlp logger: 重要メッセージはコンソール、詳細はファイルに保存"""
    def __init__(self, shard_id: int, vid: str, kind: str, client: str, log_dir: str):
        self.shard_id = shard_id
        self.vid = vid
        self.kind = kind      # "audio" or "chat"
        self.client = client  # "tv" / "android" / "web"
        self.log_dir = log_dir
        os.makedirs(log_dir, exist_ok=True)
        self.log_path = os.path.join(log_dir, f"yt_{vid}_{kind}-{client}.log")
        self._fp = open(self.log_path, "a", encoding="utf-8")
        self._printed = set()

    def _write_file(self, level: str, msg: str):
        ts = time.strftime("%Y-%m-%d %H:%M:%S")
        self._fp.write(f"[{ts}] [{level}] {msg}\n")
        self._fp.flush()

    def _maybe_print(self, level: str, msg: str):
        m = msg.lower()
        important = (
            "sabr streaming" in m or "missing a url" in m or
            "too many requests" in m or "429" in m or
            "forbidden" in m or "403" in m or
            "sign in to confirm you’re not a bot" in m or "captcha" in m or
            "timed out" in m or "timeout" in m or
            "unable to download video subtitles for 'live_chat'" in m or
            "unable to download video subtitles for 'live_chat_replay'" in m
        )
        if important:
            key = (level, msg.strip())
            if key not in self._printed:
                print(f"[{level} s{self.shard_id}] {msg}")
                self._printed.add(key)
            if "sabr streaming" in m and self.shard_id not in _SABR_HINT_PRINTED_FOR_SHARDS:
                print(f"[INFO s{self.shard_id}] SABR検知: 直リンク抑制/速度制限の可能性。"
                      f" tv→android→web 切替、non-HLS優先、低並列＆休止長めで回避中。詳細は logs/ を確認。")
                _SABR_HINT_PRINTED_FOR_SHARDS.add(self.shard_id)

    def debug(self, msg):   # 大量なのでファイルのみ
        self._write_file("DEBUG", str(msg))

    def warning(self, msg):
        s = str(msg)
        self._write_file("WARN", s)
        self._maybe_print("WARN", s)

    def error(self, msg):
        s = str(msg)
        self._write_file("ERROR", s)
        print(f"[yt-dlp ERROR s{self.shard_id}] {s}", file=sys.stderr)

    def close(self):
        try:
            self._fp.close()
        except Exception:
            pass


# ------------- progress hook (表示あり) -------------
class FancyProgressHook:
    """
    yt-dlp の progress_hooks 用
    - 一行でパーセント/サイズ/速度/ETA を更新
    - トータル不明のときは DLサイズと速度のみ
    """
    def __init__(self, shard_id: int, vid: str, kind: str, client: str, throttle: float = 0.2):
        self.shard_id = shard_id
        self.vid = vid
        self.kind = kind
        self.client = client
        self._last = 0.0
        self._throttle = throttle
        self._finished_printed = False

    def __call__(self, d: dict):
        st = d.get('status')
        now = time.time()
        if st == 'downloading':
            if now - self._last < self._throttle:
                return
            self._last = now
            dl = d.get('downloaded_bytes')
            total = d.get('total_bytes') or d.get('total_bytes_estimate')
            spd = d.get('speed')
            eta = d.get('eta')
            p = _pct(dl, total)
            if p is not None:
                line = (f"[dl s{self.shard_id}] {self.kind}@{self.client} {self.vid} | "
                        f"{p:6.2f}% {_fmt_bytes(dl)}/{_fmt_bytes(total)} "
                        f"| {_fmt_bytes(spd)}/s | ETA {_fmt_time(eta)}")
            else:
                line = (f"[dl s{self.shard_id}] {self.kind}@{self.client} {self.vid} | "
                        f"{_fmt_bytes(dl)} | {_fmt_bytes(spd)}/s")
            print("\r" + line + " " * 8, end="", flush=True)
        elif st == 'finished':
            if not self._finished_printed:
                print("\r[progress] finished → 再整理解放中..." + " " * 24)
                self._finished_printed = True


# ------------- yt-dlp downloaders -------------
def base_common_opts(outtmpl: str, hook) -> dict:
    opts = {
        "quiet": True,
        "noplaylist": True,
        "progress_hooks": [hook],
        "retries": 12,
        "fragment_retries": 12,
        "socket_timeout": 45,
        "http_chunk_size": 10 * 1024 * 1024,
        "concurrent_fragment_downloads": 1,
        "prefer_ipv4": True,
        "nocheckcertificate": True,
        "outtmpl": outtmpl,
        "no_warnings": False,
        "sleep_requests": BASE_SLEEP_REQUESTS,
        "max_sleep_requests": BASE_MAX_SLEEP_REQ,
        "ratelimit": RATE_LIMIT_BYTES,
        "overwrites": False,
        "nooverwrites": True,
    }
    if FFMPEG_PATH:
        opts["ffmpeg_location"] = os.path.dirname(FFMPEG_PATH)
    return opts

def download_audio_m4a(vid: str, url: str, out_dir: str, cookies: str,
                       shard_id: int, logs_dir: str) -> Tuple[bool, str]:
    """
    SABRを避けつつ non-HLS m4a を狙って取得。順番: tv → android → web
    """
    outtmpl = os.path.join(out_dir, "%(id)s.%(ext)s")
    FMT = "140/140-drc/bestaudio[ext=m4a][protocol!=m3u8][vcodec=none]/bestaudio[protocol!=m3u8][vcodec=none]"
    base_opts = base_common_opts(outtmpl, hook=None)  # hook は後で差し込む
    base_opts.update({
        "format": FMT,
        "writeinfojson": True,
        "ffmpeg_location": which(FFMPEG_PATH) or None,
        "prefer_ffmpeg": True,
    })
    if cookies and os.path.exists(cookies):
        base_opts["cookiefile"] = cookies

    attempts = [
        ("tv", {
            "User-Agent": "Mozilla/5.0 (SMART-TV; Linux; Tizen 2.4) AppleWebKit/538.1 (KHTML, like Gecko) SmartTV Safari/538.1",
            "Accept-Language": "ja-JP,ja;q=0.9,en-US;q=0.8,en;q=0.7",
            "Referer": url, "Sec-Fetch-Mode": "navigate",
        }),
        ("android", {
            "User-Agent": "com.google.android.youtube/19.18.39 (Linux; U; Android 13) gzip",
            "Accept-Language": "ja-JP,ja;q=0.9,en-US;q=0.8,en;q=0.7",
            "Referer": url,
        }),
        ("web", {
            "User-Agent": ("Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) "
                           "AppleWebKit/605.1.15 (KHTML, like Gecko) Version/18.4 Safari/605.1.15"),
            "Accept-Language": "ja-JP,ja;q=0.9,en-US;q=0.8,en;q=0.7",
            "Referer": url,
        }),
    ]

    last_err = "unknown"
    for client, headers in attempts:
        hook = FancyProgressHook(shard_id, vid, "audio", client)
        opts = dict(base_opts)
        opts["progress_hooks"] = [hook]
        opts["http_headers"] = headers
        opts["extractor_args"] = {"youtube": {"player_client": [client]}}

        lg = DualLogger(shard_id, vid, "audio", client, logs_dir)
        opts["logger"] = lg

        try:
            with yt_dlp.YoutubeDL(opts) as ydl:
                ydl.download([url])
            lg.close()
            print()  # 進捗行の改行
            return True, f"{client}/non-HLS"
        except (DownloadError, ExtractorError) as e:
            last_err = f"{client}: {e}"
            print()  # 進捗行の改行
        except Exception as e:
            last_err = f"{client}: {repr(e)}"
            print()
        finally:
            # 簡易診断
            try:
                probe_opts = dict(base_opts)
                probe_opts["skip_download"] = True
                with yt_dlp.YoutubeDL(probe_opts) as ydl:
                    info = ydl.extract_info(url, download=False)
                fmts = info.get("formats") or []
                nonhls = sum(1 for f in fmts if (f.get("vcodec") in (None, "none")) and "m3u8" not in (f.get("protocol") or ""))
                hls    = sum(1 for f in fmts if "m3u8" in (f.get("protocol") or ""))
                itag140 = any(str(f.get("format_id")) == "140" and f.get("url") for f in fmts)
                print(f"[diag s{shard_id}] audio@{client}: total={len(fmts)} nonHLS={nonhls} HLS={hls} itag140_url={1 if itag140 else 0}")
            except Exception:
                pass
            lg.close()

    return False, last_err


def download_livechat_json(vid: str, url: str, out_dir: str, cookies: str,
                           shard_id: int, logs_dir: str) -> Tuple[bool, str]:
    """
    live_chat / live_chat_replay を json3（なければ json）で取得。
    失敗（JSON空など）の場合は client を切替えて再試行（web→android）
    """
    outtmpl = os.path.join(out_dir, "%(id)s")

    def try_once(player: str) -> None:
        hook = FancyProgressHook(shard_id, vid, "chat", player)
        opts = base_common_opts(outtmpl, hook)
        opts.update({
            "skip_download": True,
            "writesubtitles": True,
            "writeautomaticsub": False,
            "subtitleslangs": ["live_chat", "live_chat_replay"],
            "subtitlesformat": "json3",
            "extractor_args": {"youtube": {"player_client": [player]}},
            "http_headers": {
                "User-Agent": ("Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) "
                               "AppleWebKit/605.1.15 (KHTML, like Gecko) Version/18.4 Safari/605.1.15")
                               if player == "web" else
                               ("com.google.android.youtube/19.18.39 (Linux; U; Android 13) gzip"
                                if player == "android" else
                                "Mozilla/5.0 (SMART-TV; Linux; Tizen 2.4) AppleWebKit/538.1 (KHTML, like Gecko) SmartTV Safari/538.1"),
                "Accept-Language": "ja-JP,ja;q=0.9,en-US;q=0.8,en;q=0.7",
                "Referer": url,
            },
            "ffmpeg_location": which(FFMPEG_PATH) or None,
            "prefer_ffmpeg": True,
        })
        if cookies and os.path.exists(cookies):
            opts["cookiefile"] = cookies

        lg = DualLogger(shard_id, vid, "chat", player, logs_dir)
        opts["logger"] = lg
        try:
            with yt_dlp.YoutubeDL(opts) as ydl:
                ydl.download([url])
            print()  # 進捗行の改行
        finally:
            lg.close()

    try:
        try_once("web")
        return True, "live_chat(.json3/.json)"
    except Exception as e1:
        if "Expecting value: line 1 column 1" in str(e1):
            time.sleep(3)
        try:
            try_once("android")
            return True, "live_chat via android"
        except Exception as e2:
            return False, f"{e1} | {e2}"


# ---------- duration check & auto-repair ----------
def maybe_repair_duration(vid: str, audio_dir: str, meta_dir: str) -> None:
    if not DURATION_CHECK_ENABLED:
        return
    src = find_audio_file(audio_dir, vid)
    if not src:
        return
    info = find_infojson(audio_dir, meta_dir, vid)
    exp = read_expected_duration_from_infojson(info) if info else None
    act = ffprobe_duration_seconds(src, FFPROBE_PATH)
    if needs_repair(exp, act):
        tmp_out = os.path.join(audio_dir, f"{vid}.fixed.m4a")
        ok, msg = try_repair_container_ffmpeg(src, tmp_out, FFMPEG_PATH)
        if ok and os.path.exists(tmp_out):
            try:
                os.replace(tmp_out, src)
                print(f"  [FIX] duration mismatch (expected={exp}s, actual={act}s) → remux OK")
            except Exception as e:
                print(f"  [WARN] remux ok but replace failed: {e}")
        else:
            print(f"  [WARN] remux failed ({msg}). exp={exp}s actual={act}s")
    else:
        if exp is not None and act is not None:
            print(f"  [len] expected~{int(exp)}s / actual~{int(act)}s (OK)")


# ------------- worker entry -------------
def run_worker_one(RUN_ID: str, SHARD_ID: int, COOKIES: str, limit: int | None = None):
    base = run_root(RUN_ID)
    paths = ensure_dirs(base)
    assign_csv = os.path.join(paths["mani"], f"assignments_audio_{RUN_ID}.csv")
    if not os.path.exists(assign_csv):
        print("ERROR: assignments が見つかりません:", assign_csv, file=sys.stderr)
        return 2

    if not COOKIES or not os.path.exists(COOKIES):
        print("ERROR: cookies.txt が見つかりません:", COOKIES, file=sys.stderr)
        return 3

    if not cookie_is_fresh(COOKIES, COOKIE_MAX_AGE_DAYS):
        print(f"[WARN] cookie が古い可能性（{COOKIE_MAX_AGE_DAYS}日超）。失敗が多い場合は再エクスポートを。\n  {COOKIES}")

    targets: List[str] = []
    with open(assign_csv, "r", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            try:
                sid = int(row["shard_id"])
            except Exception:
                continue
            if sid == SHARD_ID:
                vid = row["video_id"].strip()
                if re.fullmatch(r"^[\w-]{11}$", vid):
                    targets.append(vid)

    if limit:
        targets = targets[:limit]

    total = len(targets)
    print(f"[shard {SHARD_ID}] COUNT={total}  OUT_AUDIO={paths['audio']}  OUT_CHAT={paths['chat']}  META={paths['meta']}")

    ok_audio = ok_chat = 0
    skip_audio = skip_chat = 0
    fail_audio = fail_chat = 0

    for idx, vid in enumerate(targets, 1):
        url = normalize_to_url(vid)
        print(f"\n=== [s{SHARD_ID} {idx}/{total}] {vid} ===")

        # Audio
        if already_has_audio(paths["audio"], vid):
            print("  ↪ audio: skip (exists)")
            skip_audio += 1
        else:
            a_ok, a_msg = download_audio_m4a(vid, url, paths["audio"], COOKIES, SHARD_ID, paths["logs"])
            if a_ok:
                print("  ✅ audio OK:", a_msg)
                try:
                    maybe_repair_duration(vid, paths["audio"], paths["meta"])
                except Exception:
                    print("  [WARN] duration check failed:\n", traceback.format_exc())
                try:
                    move_info_json_to_meta(paths["audio"], paths["meta"])
                except Exception:
                    print("  [WARN] move info.json failed:\n", traceback.format_exc())
                ok_audio += 1
            else:
                print("  ❌ audio NG:", a_msg)
                fail_audio += 1

        # Chat
        if already_has_chat(paths["chat"], vid):
            print("  ↪ chat : skip (exists)")
            skip_chat += 1
        else:
            try:
                c_ok, c_msg = download_livechat_json(vid, url, paths["chat"], COOKIES, SHARD_ID, paths["logs"])
            except Exception as e:
                c_ok, c_msg = False, repr(e)
            if c_ok:
                print("  ✅ chat  OK:", c_msg)
                ok_chat += 1
            else:
                print("  ❌ chat  NG:", c_msg)
                fail_chat += 1

        time.sleep(SLEEP_BETWEEN_VIDEOS)

    print(f"\n[shard {SHARD_ID}] SUMMARY  audio: ok={ok_audio} skip={skip_audio} fail={fail_audio} | "
          f"chat: ok={ok_chat} skip={skip_chat} fail={fail_chat}")
    print("out :", base)
    return 0


def main():
    run_dir = run_root(RUN_ID)
    print(f"RUN_ID={RUN_ID}  RUN_DIR={run_dir}  SHARD_IDS={SHARD_IDS}  PARALLEL={PARALLEL}")
    print(f"COOKIES={COOKIES}  (max_age_days={COOKIE_MAX_AGE_DAYS})")

    if not os.path.isdir(run_dir):
        print("ERROR: run_dir が存在しません。プランナーを先に実行してください。", file=sys.stderr)
        sys.exit(2)

    found_ffmpeg  = which(FFMPEG_PATH)  or shutil.which("ffmpeg")
    found_ffprobe = which(FFPROBE_PATH) or shutil.which("ffprobe")
    if not found_ffmpeg:
        print("[WARN] ffmpeg が見つかりません。m4a のタイムスタンプ修復ができない可能性があります。")
    if DURATION_CHECK_ENABLED and not found_ffprobe:
        print("[WARN] ffprobe が見つかりません。実長の自動検査が無効になります。")

    if PARALLEL and len(SHARD_IDS) > 1:
        from multiprocessing import Process
        procs = []
        for sid in SHARD_IDS:
            p = Process(target=run_worker_one, args=(RUN_ID, int(sid), COOKIES))
            p.start()
            procs.append(p)
            time.sleep(3 * int(sid))
        for p in procs:
            p.join()
    else:
        for sid in SHARD_IDS:
            run_worker_one(RUN_ID, int(sid), COOKIES)

if __name__ == "__main__":
    try:
        main()
    except Exception as e:
        print("Fatal:", repr(e), file=sys.stderr)
        sys.exit(3)


